In [42]:
"""
Step 1: Import necessary libraries
I have moved the iterations to the very end
"""
import re
from pathlib import Path
import pandas as pd

In [43]:
"""
Step 2: Extract all necessary information from the 10-k files
"""
def clean_html(raw_html):
    """
    This function is used to clean the raw content
    """
    # remove html tags
    clean_one = re.sub(r'<.*?>', ' ', raw_html)
    # remove html entities
    clean_two = re.sub(r'&\w+;', ' ', clean_one)
    clean_two = re.sub(r'&nbsp;', ' ', clean_two)
    clean_two = re.sub(r'&amp;', '&', clean_two)

    clean_three = re.sub(r"&[a-z]+;", " ", clean_two)

    # remove multiple spaces
    clean_four = re.sub(r'\s+', ' ', clean_three)
    return clean_four.strip()

In [47]:
"""
Step 3: Extract all necessary information from the 10-k files
"""
# fiscal year不仅要满足年份的格式，还要满足年份的范围，要去文中查相应字段
def extract_fiscal_year(text):
    """ 1. The date of the fiscal year-end (ensure it is formatted in ISO-format)"""
    fiscal_year_end = re.search(r"fiscal year ended (\d{4}-\d{2}-\d{2})", text, re.IGNORECASE).group(1)
    return fiscal_year_end

def extract_legal_proceedings(text):
    """ 2. The content of “Item 3. LEGAL PROCEEDINGS”."""
    # TODO: Not sure which implementation is correct
    legal_proceedings = re.search(r"Item 3. LEGAL PROCEEDINGS(.*?)Item 4", text, re.IGNORECASE|re.DOTALL).group(1)

    # legal_proceedings = re.search(r"Item 3\. Legal Proceedings(.*?)(Item 4\.|$)", text, re.DOTALL).group(1).strip()
    return legal_proceedings

def extract_signature_date(text):
    """ 3. The date of signature(s) (ensure it is formatted in ISO-format)."""
    # TODO: Not sure which implementation is correct
    # signature_date = re.search(r"SIGNATURES(.*?)</html>", text, re.IGNORECASE|re.DOTALL).group(1)

    signature_date = re.search(r"Date:\s*(\d{4}-\d{2}-\d{2})", text).group(1)

    return signature_date

def extract_signatures(text):
    """ 4. Who signed the report? If there are multiple signatures, all of them have to be listed (comma separated).
    Do not include the audit firm – in case it is given in the report."""
    # TODO: Not sure which implementation is correct
    signers = re.findall(r"(?:By|Signed by):\s*([A-Za-z ]+)\n", text)
    # signers = ", ".join([s.strip() for s in signers if "audit" not in s.lower()])

    signatures = re.findall(r"SIGNATURES(.*?)</html>", text, re.IGNORECASE|re.DOTALL)
    return signatures

In [48]:
"""
Step 4: Process the whole file
"""
def process_file(file):
    """ This function processes a single 10-k file and extracts the necessary information """
    clean_content = clean_html(file)

    fiscal_year = extract_fiscal_year(clean_content)

    legal_proceedings = extract_legal_proceedings(clean_content)

    signature_date = extract_signature_date(clean_content)

    signers = extract_signatures(clean_content)
    return {
        "fiscal_year": fiscal_year,
        "legal_proceedings": legal_proceedings,
        "signature_date": signature_date,
        "signers": signers
    }

In [49]:
"""
Step 5: Process all the files and store the results in a DataFrame
"""
current_dictionary = Path.cwd()
# TODO: Path.cwd() or Path('A1/10-k html files')
for html_file in current_dictionary.glob('*.html'):
    # TODO: utf-8 encoding or ISO-8859-1 encoding or ASCII?
    with html_file.open('r', encoding='utf-8') as file:
        # print(file.read()[0:100])
        # read or readlines? Which one is more space efficient?
        raw_content = file.read()
        results = process_file(raw_content)
        # TODO: call process file if needed, do we need if __name__ == '__main__'? in jupyter notebook
        # print(raw_content[0:100])
        # print('below are cleaned content')
        # print(clean_html(raw_content)[0:100])

# store the results in a pandas DataFrame
df = pd.DataFrame(results)
df.to_csv('10k_results.csv', index=False)
df.head()

AttributeError: 'NoneType' object has no attribute 'group'

In [51]:
import re
from pathlib import Path
import pandas as pd

def clean_html(raw_html):
    clean_one = re.sub(r'<.*?>', ' ', raw_html)
    clean_two = re.sub(r'&\w+;', ' ', clean_one)
    clean_two = re.sub(r'&nbsp;', ' ', clean_two)
    clean_two = re.sub(r'&amp;', '&', clean_two)
    clean_three = re.sub(r"&[a-z]+;", " ", clean_two)
    clean_four = re.sub(r'\s+', ' ', clean_three)
    return clean_four.strip()

def extract_fiscal_year(text):
    match = re.search(r"fiscal year ended (\d{4}-\d{2}-\d{2})", text, re.IGNORECASE)
    return match.group(1) if match else 'N/A'

def extract_legal_proceedings(text):
    match = re.search(r"Item 3\. Legal Proceedings(.*?)(Item 4\.|$)", text, re.IGNORECASE|re.DOTALL)
    return match.group(1).strip() if match else 'N/A'

def extract_signature_date(text):
    match = re.search(r"Date:\s*(\d{4}-\d{2}-\d{2})", text)
    return match.group(1) if match else 'N/A'

def extract_signatures(text):
    signers = re.findall(r"(?:By|Signed by):\s*([A-Za-z ]+)\n", text)
    signers = ", ".join([s.strip() for s in signers if "audit" not in s.lower()])
    return signers if signers else 'N/A'

def process_file(file):
    clean_content = clean_html(file)
    fiscal_year = extract_fiscal_year(clean_content)
    legal_proceedings = extract_legal_proceedings(clean_content)
    signature_date = extract_signature_date(clean_content)
    signers = extract_signatures(clean_content)
    return {
        "fiscal_year": fiscal_year,
        "legal_proceedings": legal_proceedings,
        "signature_date": signature_date,
        "signers": signers
    }

current_directory = Path.cwd()
results = []
for html_file in current_directory.glob('*.html'):
    with html_file.open('r', encoding='utf-8') as file:
        raw_content = file.read()
        result = process_file(raw_content)
        if result:
            results.append(result)

df = pd.DataFrame(results)
df.to_csv('10k_results.csv', index=False)
df.head()

,fiscal_year,legal_proceedings,signature_date,signers
0,N/A,","" under the sub-caption ""I. Supplemental Info...",N/A,N/A
1,N/A,25,N/A,N/A
2,N/A,The information called for by this item is inc...,N/A,N/A
3,N/A,18,N/A,N/A
4,N/A,18,N/A,N/A
